In [ ]:
#!python -m spacy download en_core_web_sm
#!pip3 install -U pywsd
import nltk
nltk.download('omw-1.4')
!pip3 install -U pywsd
nltk.download('averaged_perceptron_tagger')
# import packages

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import string
import re
import spacy

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
# from pywsd.utils import lemmatize_sentence

import itertools
from tqdm.auto import tqdm
tqdm.pandas()
from multiprocessing import Pool

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# settings
import multiprocessing
text_cols = ['text']
cores = multiprocessing.cpu_count()


In [ ]:
url = 'https://git.uwaterloo.ca/t4shan/cs-686-sentiment-analysis/-/raw/main/training.1600000.processed.noemoticon.csv'
columns = ["target", "ids", "date", "flag", "user", "text"]
df = pd.read_csv(url, encoding = "ISO-8859-1", names=columns)

In [ ]:
len(df)
# all column names
print(df.columns.values)
# first 5 rows
df.head()
# check possible value for the column
df.flag.value_counts()

['target' 'ids' 'date' 'flag' 'user' 'text']


NO_QUERY    1600000
Name: flag, dtype: int64

Use a smaller sample size - (1/100 of the original dataset).
Then, we only keep columns that are useful eg. target(0 = negative, 2 = neutral, 4 = positive) and text.

Then, we turn column name to label and text, where label takes negative as 0 and everything else as 1.

In [ ]:
df_small = df.sample(n = int(len(df)/100), replace = False)
# df_subset = df_small[['target', 'text']]
# df_subset.columns = ['label', 'text']
# # keep 0 as 0 and change everything else as 1
# df_subset.loc[df_subset['label'] > 0, 'label'] = 1

label_text = df_small[['target', 'text']]
# Convert labels to range 0-1                                        
label_text['target'] = label_text['target'].apply(lambda x: 0 if x == 0 else 1)

# Assign proper column names to labels
label_text.columns = ['label', 'text']

# Assign proper column names to labels
label_text.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,label,text
846379,1,Is glad to be back at work
834154,1,@maron101 is that what we're having for breakf...
229457,0,A lazy lazy Sunday. I miss thrift shopping.
1154261,1,@Peaso thnx facebook is still the best !!..
1223737,1,"@BenedicteBa Of course, it's an amazing song. ..."


# with BertLibrary

replace hashtags and mentions with custom tokens

In [ ]:
import re

hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls = re.compile(r"https?://\S+")

def process_text(text):
  text = hashtags.sub(' hashtag', text)
  text = mentions.sub(' entity', text)
  return text.strip().lower()
  
def match_expr(pattern, string):
  return not pattern.search(string) == None

def get_data_wo_urls(dataset):
    link_with_urls = dataset.text.apply(lambda x: match_expr(urls, x))
    return dataset[[not e for e in link_with_urls]]

In [ ]:
label_text.text = label_text.text.apply(process_text)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
from sklearn.model_selection import train_test_split
TRAIN_SIZE = 0.75
VAL_SIZE = 0.05
dataset_count = len(label_text)

df_train_val, df_test = train_test_split(label_text, test_size=1-TRAIN_SIZE-VAL_SIZE, random_state=42)
df_train, df_val = train_test_split(df_train_val, test_size=VAL_SIZE / (VAL_SIZE + TRAIN_SIZE), random_state=42)

print("TRAIN size:", len(df_train))
print("VAL size:", len(df_val))
print("TEST size:", len(df_test))

TRAIN size: 12000
VAL size: 800
TEST size: 3200


In [ ]:
df_train = get_data_wo_urls(df_train)
df_train.head()

,label,text
1023268,1,entity what a wonderful idea for breakfast!
1384100,1,entity happy birthday loris
487260,0,"so many cool writer t-shirts, so little money"
1080553,1,oh sunset over the ocean and mountains
378292,0,entity this is all i wanted for m...


Putting them in files for BERT to load

In [ ]:
!mkdir dataset
df_train.sample(frac=1.0).reset_index(drop=True).to_csv('dataset/train.tsv', sep='\t', index=None, header=None)
df_val.to_csv('dataset/dev.tsv', sep='\t', index=None, header=None)
df_test.to_csv('dataset/test.tsv', sep='\t', index=None, header=None)
! cd dataset && ls

dev.tsv  test.tsv  train.tsv


https://github.com/kpi6research/Bert-as-a-Library/blob/master/README.md

In [ ]:
import tensorflow as tf
print(tf.__version__)

# !pip uninstall tensorflow==tf.__version__

!pip install tensorflow==1.15.0
!pip install tensorflow-gpu==1.15.0

1.15.0


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

''

In [ ]:
!pip install BertLibrary
from BertLibrary import BertFTModel
import numpy as np

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2022-11-30 00:39:19--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.128, 142.251.12.128, 74.125.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.1’

uncased_L-12_H-768_ 100%[===================>] 388.84M  23.6MB/s    in 18s     

2022-11-30 00:39:37 (22.1 MB/s) - ‘uncased_L-12_H-768_A-12.zip.1’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
replace uncased_L-12_H-768_A-12/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace uncased_L-12_H-768_A-12/vocab.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace uncased_L-12_H-768_A-12/bert_model.ckpt.in

In [ ]:
!mkdir output
ft_model = BertFTModel( model_dir='uncased_L-12_H-768_A-12',
                        ckpt_name="bert_model.ckpt",
                        labels=['0','1'],
                        lr=1e-05,
                        num_train_steps=30000,
                        num_warmup_steps=1000,
                        ckpt_output_dir='output',
                        save_check_steps=1000,
                        do_lower_case=False,
                        max_seq_len=50,
                        batch_size=32,
                        )


ft_trainer =  ft_model.get_trainer()
ft_evaluator = ft_model.get_evaluator()

mkdir: cannot create directory ‘output’: File exists


In [ ]:
ft_trainer.train_from_file('dataset', 35000)





Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for updating:
Use `tf.cast` instead.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instruction

KeyboardInterrupt: ignored

In [ ]:
ft_evaluator.evaluate_from_file('dataset', checkpoint="output/model.ckpt-35000") 

____________________________________________________________________________________

#BERT Model

In [ ]:
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case = True)

In [ ]:
encoder = tokenizer.batch_encode_plus(df_subset.text.values,
                                           add_special_tokens = True,
                                           pad_to_max_length = True,
                                           max_length = 256,
                                           return_tensors = 'pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
encoder

{'input_ids': tensor([[  101,  3407, 10756,  ...,     0,     0,     0],
        [  101,  1030, 17207,  ...,     0,     0,     0],
        [  101,  1030, 25353,  ...,     0,     0,     0],
        ...,
        [  101,  1030, 15315,  ...,     0,     0,     0],
        [  101,  1998,  5470,  ...,     0,     0,     0],
        [  101,  1045,  1005,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}